## Import

In [33]:
import os
import sys
import argparse
import numpy as np
import supervision as sv
from pathlib import Path
from ultralytics import YOLO
from dotenv import load_dotenv

## Path & Env

In [34]:
# path
current_path = Path(os.getcwd())
sys.path.insert(0, current_path)

In [35]:
# env
dotenv_path = f"{current_path}/.env"
load_dotenv(dotenv_path=dotenv_path, override=True) 

False

## Model

In [40]:
model = YOLO("yolov8n.pt")
box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()
tracker = sv.ByteTrack()

## Annotation

In [44]:
def process_frame(frame: np.ndarray, frame_index: int) -> np.ndarray:
    results = model(frame)[0]

    detections = sv.Detections.from_ultralytics(results)
    detections = tracker.update_with_detections(detections)

    # Build labels with class name + ID
    labels = []
    for class_id, tracker_id in zip(detections.class_id, detections.tracker_id):
        class_name = model.model.names[class_id] if class_id is not None else "unknown"
        label = f"{class_name} #{tracker_id}" if tracker_id is not None else class_name
        labels.append(label)

    # First draw boxes
    annotated_frame = box_annotator.annotate(
        scene=frame.copy(),
        detections=detections
    )

    # Then draw labels separately
    annotated_frame = label_annotator.annotate(
        scene=annotated_frame,
        detections=detections,
        labels=labels
    )

    return annotated_frame

## Main Process

In [45]:
def main(source_video_path: str):
    target_video_path = f"{current_path}/result.mp4"
    print(f"Processing video from '{source_video_path}' and saving to '{target_video_path}'...")
    sv.process_video(
        source_path=source_video_path,
        target_path=target_video_path,
        callback=process_frame
    )
    print("Video processing complete.")

In [46]:
if __name__ == "__main__":
    video_path = f"{current_path}/videos/sample3.mp4"
    main(video_path)

Processing video from '/mnt/batch/tasks/shared/LS_root/mounts/clusters/gpu-8-56-352-16-jh/code/Users/jeremy.heng/opencv_yolo_supervision/videos/sample3.mp4' and saving to '/mnt/batch/tasks/shared/LS_root/mounts/clusters/gpu-8-56-352-16-jh/code/Users/jeremy.heng/opencv_yolo_supervision/result.mp4'...

0: 384x640 3 persons, 10 cars, 1 truck, 6 traffic lights, 6.1ms
Speed: 2.0ms preprocess, 6.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 10 cars, 1 truck, 5 traffic lights, 6.0ms
Speed: 1.5ms preprocess, 6.0ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 9 cars, 1 truck, 6 traffic lights, 6.1ms
Speed: 1.0ms preprocess, 6.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 9 cars, 1 truck, 5 traffic lights, 6.0ms
Speed: 1.2ms preprocess, 6.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 1 truck, 6 traffic lights, 6.5ms
Spe